## Financial Time Series

Frameworks:
- Kedro (DS Framework)
- Prefect (Scheduler)
- Postgres (Database)

- Build node / pipelines in Kedro
- Translate to a prefect flow

Data Sources:
- Binance API

Output
- Notifications
  - Email
  - App? Flutter

Workflows:

- Tickers
  - What it is?
    - What tickers are available
    - What tickers do I want to run?
  - Database
    - Ticker, Run, Available
    - Str, bool, bool
  - Refresh Tickers
  - Change state (Run)

- Price / Volume History
  - What is is?
    - Minute by minute tracking
    - Price, Volume
    - Could look at trades?, but not for now
  - Database
    - Time, Price, Volume, Ticker
    - Datetime, float, float, str
    - How to synchronise workers, will it just work, or do I need database record that I update
    - Sync needs to be single threaded
  - Update prices


In [6]:
import requests
import pandas as pd

In [7]:
columns = [
"StartTime",
"Open",
"High",
"Low",
"Close",
"Volume",
"EndTime",
"QuoteVolume",
"NumTrades",
"BuyBaseVolume",
"BuyQuoteVolume",
"Ignore"
]

In [8]:
exchange_url = f'https://api.binance.com/api/v3/exchangeInfo'

In [9]:
exchange_response = requests.get(url = exchange_url)
exchange_data = exchange_response.json()

In [10]:
symbol_list = pd.DataFrame(exchange_data['symbols'])

In [21]:
symbol_list.loc[symbol_list.symbol.str.contains('USDT')].reset_index(drop=True)[0:20].symbol.to_list()

['BTCUSDT',
 'ETHUSDT',
 'BNBUSDT',
 'BCCUSDT',
 'NEOUSDT',
 'LTCUSDT',
 'QTUMUSDT',
 'ADAUSDT',
 'XRPUSDT',
 'EOSUSDT',
 'TUSDUSDT',
 'IOTAUSDT',
 'XLMUSDT',
 'ONTUSDT',
 'TRXUSDT',
 'ETCUSDT',
 'ICXUSDT',
 'VENUSDT',
 'NULSUSDT',
 'VETUSDT']

In [17]:
symbol = symbol_list[0]
kline_url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1h&limit=10'
r = requests.get(url = kline_url)
data = r.json()
df =  pd.DataFrame(data[0][:-1]).transpose()

In [27]:
df = pd.DataFrame(data)

In [30]:
df.columns = columns

In [31]:
df

,StartTime,Open,High,Low,Close,Volume,EndTime,QuoteVolume,NumTrades,BuyBaseVolume,BuyQuoteVolume,Ignore
0,1650880800000,0.07326300,0.07354200,0.07322000,0.07350400,3200.14660000,1650884399999,234.94193787,6239,1818.56020000,133.51066915,0
1,1650884400000,0.07350400,0.07372200,0.07318900,0.07370100,5709.52770000,1650887999999,419.48206058,9804,3394.08680000,249.39722246,0
2,1650888000000,0.07370100,0.07377500,0.07355300,0.07370100,5385.24110000,1650891599999,396.72670613,7796,2937.54440000,216.41839968,0
3,1650891600000,0.07370200,0.07390600,0.07352300,0.07376800,5043.30590000,1650895199999,371.68308509,9068,2426.33810000,178.82090972,0
4,1650895200000,0.07376800,0.07382600,0.07330800,0.07351300,6447.22370000,1650898799999,474.39804800,11151,2922.33460000,215.02337609,0
5,1650898800000,0.07351300,0.07366000,0.07341300,0.07362500,4633.88150000,1650902399999,340.75886414,7391,2241.93920000,164.86106845,0
6,1650902400000,0.07362400,0.07408100,0.07345000,0.07408000,6964.52960000,1650905999999,514.13989082,11808,3675.25090000,271.33335871,0
7,1650906000000,0.07408000,0.07434300,0.07406900,0.07431600,4183.50270000,1650909599999,310.66777187,8822,2394.65350000,177.80961393,0
8,1650909600000,0.07431800,0.07453100,0.07388800,0.07422700,7265.44770000,1650913199999,539.46560380,12182,4263.19040000,316.59488464,0
9,1650913200000,0.07422600,0.07442200,0.07403000,0.07436400,4828.07300000,1650916799999,358.27206111,7160,2845.47380000,211.16157960,0


In [35]:
df['StartTime'] = pd.to_datetime(df['StartTime'], utc=True, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')
df['EndTime'] = pd.to_datetime(df['EndTime'], utc=True, unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')

In [37]:
df

,StartTime,Open,High,Low,Close,Volume,EndTime,QuoteVolume,NumTrades,BuyBaseVolume,BuyQuoteVolume,Ignore
0,2022-04-25 10:00:00,0.07326300,0.07354200,0.07322000,0.07350400,3200.14660000,2022-04-25 10:59:59,234.94193787,6239,1818.56020000,133.51066915,0
1,2022-04-25 11:00:00,0.07350400,0.07372200,0.07318900,0.07370100,5709.52770000,2022-04-25 11:59:59,419.48206058,9804,3394.08680000,249.39722246,0
2,2022-04-25 12:00:00,0.07370100,0.07377500,0.07355300,0.07370100,5385.24110000,2022-04-25 12:59:59,396.72670613,7796,2937.54440000,216.41839968,0
3,2022-04-25 13:00:00,0.07370200,0.07390600,0.07352300,0.07376800,5043.30590000,2022-04-25 13:59:59,371.68308509,9068,2426.33810000,178.82090972,0
4,2022-04-25 14:00:00,0.07376800,0.07382600,0.07330800,0.07351300,6447.22370000,2022-04-25 14:59:59,474.39804800,11151,2922.33460000,215.02337609,0
5,2022-04-25 15:00:00,0.07351300,0.07366000,0.07341300,0.07362500,4633.88150000,2022-04-25 15:59:59,340.75886414,7391,2241.93920000,164.86106845,0
6,2022-04-25 16:00:00,0.07362400,0.07408100,0.07345000,0.07408000,6964.52960000,2022-04-25 16:59:59,514.13989082,11808,3675.25090000,271.33335871,0
7,2022-04-25 17:00:00,0.07408000,0.07434300,0.07406900,0.07431600,4183.50270000,2022-04-25 17:59:59,310.66777187,8822,2394.65350000,177.80961393,0
8,2022-04-25 18:00:00,0.07431800,0.07453100,0.07388800,0.07422700,7265.44770000,2022-04-25 18:59:59,539.46560380,12182,4263.19040000,316.59488464,0
9,2022-04-25 19:00:00,0.07422600,0.07442200,0.07403000,0.07436400,4828.07300000,2022-04-25 19:59:59,358.27206111,7160,2845.47380000,211.16157960,0


In [ ]:
df[['Open','High','Low','Close','Volume','QuoteVolume','BuyBaseVolume','BuyQuoteVolume']]= df[['Open','High','Low','Close','Volume','QuoteVolume','BuyBaseVolume','BuyQuoteVolume']].apply(pd.to_numeric)
   

In [40]:
df['VaisMagicMetric'] = (df['Open'] - df['Close'] )/ df['Volume'] 

In [42]:
df.to_csv('abc.csv')

In [84]:
df.insert(loc = 0,
          column = 'Symbol',
          value = symbol)

In [85]:
data = tuple(df.loc[0])

In [1]:
import psycopg2 as pg
from dotenv import load_dotenv
import os 
load_dotenv('./.env')

True

In [87]:
db_name = os.getenv('DATABASE')
host = os.getenv('HOST')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
port = os.getenv('PORT')

In [101]:
conn = pg.connect(
    dbname=db_name,
    user=user,
    password=password,
    host=host,
    port=port,
)

query = """
    INSERT INTO kline
    (symbol, starttime, open, high, low, close, volume, endtime, quotevolume, numtrades, buybasevolume, buyquotevolume)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """

# try to execute query
# context manager automatically rolls back failed transactions
try:
    cursor = conn.cursor()
    cursor.execute(query=query, vars=data)
    conn.commit()
    
# ensure connection is closed
finally:
    conn.close()


In [2]:
"""
This is a boilerplate pipeline 'notifications'
generated using Kedro 0.17.7
"""

from dotenv import load_dotenv
import os 
import smtplib
import time


load_dotenv()



In [5]:
send_notification('hello')

hello
None None


SMTPServerDisconnected: please run connect() first